In [ ]:
import logging; logging.basicConfig(level=logging.DEBUG)
import time
import uuid
import pprint
import datetime as dt
import json
import os

import dropbot
from dropbot import SerialProxy
from ipywidgets import widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from dropbot.hardware_test import *

%matplotlib inline

try:
    proxy.terminate()
except:
    pass

# Connect to the connected DropBot and print it's properties
proxy = SerialProxy()


def plot_and_fit(x, y, order=1):
    plt.plot(x, y, 'o')
    p = np.polyfit(x, y, order)
    x = np.linspace(x[0], x[-1], 10)
    y = 0
    for i in range(order + 1):
        y += x**(order-i) * p[i]
    plt.plot(x, y, 'k--')
    return p


def display_system_info_results(info):    
    print '\nControl board:\n' + '-' * 80
    
    print '\n  uuid:', info['control board']['uuid']
    print '\n  Properties:'
    for k, v in info['control board']['properties'].items(): print '    %s: %s' % (k, v)   
    print '\n  Config:'
    for k, v in info['control board']['config'].items(): print '    %s: %s' % (k, v)

    print '\nsoft i2c scan:', info['soft_i2c_scan']
    print 'number_of_channels:', info['number_of_channels']        


def display_test_i2c_results(results):
    print '\ni2c scan:\n' + '-' * 80
    
    for address in sorted(results['i2c_scan']):
        data = results['i2c_scan'][address]
        info_string = ''
        if 'name' in data.keys():
            info_string = data['name']
        if 'hardware_version' in data.keys():
            info_string += " v%s" % data['hardware_version']
        if 'software_version' in data.keys():
            info_string += ", firmware: v%s" % data['software_version']
        if 'uuid' in data.keys():
            info_string += ", uuid: %s" % data['uuid']

        print '  %s: %s' % (address, info_string)    
    
    
def display_test_voltage_results(results):
    print '\nTest voltage results:\n' + '-' * 80
    
    measured_voltage = np.array(results['measured_voltage'])
    target_voltage = np.array(results['target_voltage'])

    print '  target_voltage:', target_voltage
    print '  measured_voltage:', measured_voltage

    # calculate the average rms error
    r = measured_voltage - target_voltage
    print '  rms_error = %.1f%%' % (100 * np.sqrt(np.mean((r / target_voltage)**2)))
    
    plt.figure()
    # plot the measured vs target votage
    plt.plot(target_voltage, measured_voltage, 'o')
    plt.plot(target_voltage, target_voltage, 'k--')
    plt.xlabel('Target voltage')
    plt.ylabel('Measured voltage')
    
    plt.figure()
    p = plot_and_fit(results['target_voltage'],
                     results['input_current'], 2)
    print '  input_current vs target_voltage fit: p = %s' % p
    plt.xlabel('Target voltage')
    plt.ylabel('Input current (A)')

    plt.figure()
    p = plot_and_fit(results['target_voltage'],
                     np.array(results['output_current']))
    print '  output_current vs target_voltage fit: p = %s' % p
    plt.xlabel('Target voltage')
    plt.ylabel('Output current (A)')    
    
    
def display_test_on_board_feedback_calibration_results(results):
    c_measured = np.array(results['c_measured'])
    
    print '\nTest on-board feedback calibration results:\n' + '-' * 80

    print '  Measured capacitance:', c_measured
    
    C_nominal = np.array([0, 10e-12, 100e-12, 470e-12])
    plt.figure()
    plt.plot(C_nominal * 1e12, c_measured * 1e12, 'o')
    plt.plot(C_nominal * 1e12, C_nominal * 1e12, 'k--')
    plt.xlabel('Nominal capacitance (pF)')
    plt.ylabel('Measured capacitance (pF)')
    
    
def display_test_shorts_results(results):
    shorts = results['shorts']
    print '\nTest shorts results:\n' + '-' * 80
    if len(shorts):
        print "  Shorts on channels %s" % ", ".join([str(x) for x in shorts])
    else:
        print "  No shorts"
        
        
def display_test_channels_results(results):
    print '\nTest channels results:\n' + '-' * 80
    
    c = np.array(results['c'])
    test_channels = np.array(results['test_channels'])
    shorts = results['shorts']
    n_channels = len(test_channels)
    n_reps = c.shape[1]
    
    if len(c) == 0:
        return
    
    nc = test_channels[np.min(c, 1) < 5e-12].tolist()
    for x in shorts:
        nc.remove(x)
    
    plt.figure()
    plt.bar(range(c.shape[0]), np.mean(c, 1) / 1e-12, yerr=np.std(c, 1) / 1e-12)
    plt.title('DropBot system: %s' % str(proxy.uuid)[-8:])
    plt.xlabel("Channel");
    plt.ylabel("Capacitance (pF)")

    plt.figure()
    plt.hist(c.flatten() / 1e-12, 20)
    plt.ylabel("# of channels")
    plt.xlabel("Capacitance (pF)")

    if len(shorts) or len(nc):
        print "  The following channels failed (%d of %d / %.1f %%):" % (len(shorts) + len(nc),
                                                                        n_channels,
                                                                        float(len(shorts) + len(nc)) /
                                                                             n_channels * 100)
        if len(shorts):
            print "    shorts (%d of %d / %.1f %%): %s" % (len(shorts), n_channels,
                                                             float(len(shorts)) / n_channels * 100,
                                                             ", ".join([str(x) for x in shorts]))                                                
        if len(nc):
            print "    no connection (%d of %d / %.1f %%): %s" % (len(nc), n_channels,
                                                             float(len(nc)) / n_channels * 100,
                                                             ", ".join([str(x) for x in nc]))
            if n_reps > 1:
                for x in nc:
                    n_fails = np.count_nonzero(c[x, :] < 5e-12)
                    print "\n    Channel %d failed %d of %d reps (%.1f %%)" % (x, n_fails, n_reps, 100.0 * n_fails / n_reps)
    else:
        print "  All channels passed"

In [ ]:
# perform QC tests
total_time = 0

tests = ['system_info',
         'test_i2c',
         'test_voltage',
         'test_shorts',
         'test_on_board_feedback_calibration',
         'test_channels'
        ]
results = {}

for test in tests:
    start_time = time.time()
    args = ''
    if test == 'test_channels':
        args = ', n_reps = 1'
    exec('results["%s"] = %s(proxy%s)' % (test, test, args))        
    results[test]['utc_timestamp'] = dt.datetime.utcnow().isoformat()
    duration = time.time() - start_time
    results[test]['test_duration'] = duration
    print "%s: %.1f s" % (test, duration)
    total_time += duration

print "-" * 80
print "Total time: %.1f s\n" % total_time

# need to create a custom encoder to serialize numpy datatypes
class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.uint8):
            return float(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)

        
# write the results to a file
filepath = ('.results/results-%s.json' % 
            dt.datetime.utcnow().isoformat().replace(':', '.'))
if not os.path.exists('.results'):
    os.mkdir('.results')
with open(filepath, 'w') as output:
    json.dump(results, output, cls=MyEncoder)
    
print 'Results saved to "%s"' % filepath

In [ ]:
# load the results from the file
print 'Loading results from "%s"...\n' % filepath
with open(filepath, 'r') as input:
    results = json.load(input)

# display the results of all QC tests
for test in tests:
    exec('display_%s_results(results["%s"])' % (test, test))